In [1]:
import pandas as pd
import numpy as np

In [2]:
# Create the "sales" table
sales_data = [
    ('A', '2021-01-01', '1'),
    ('A', '2021-01-01', '2'),
    ('A', '2021-01-07', '2'),
    ('A', '2021-01-10', '3'),
    ('A', '2021-01-11', '3'),
    ('A', '2021-01-11', '3'),
    ('B', '2021-01-01', '2'),
    ('B', '2021-01-02', '2'),
    ('B', '2021-01-04', '1'),
    ('B', '2021-01-11', '1'),
    ('B', '2021-01-16', '3'),
    ('B', '2021-02-01', '3'),
    ('C', '2021-01-01', '3'),
    ('C', '2021-01-01', '3'),
    ('C', '2021-01-07', '3')
]

sales_columns = ['customer_id', 'order_date', 'product_id']
sales_df = pd.DataFrame(sales_data, columns = sales_columns)



# Create the "menu" table
menu_data = [
    ('1', 'sushi', 10),
    ('2', 'curry', 15),
    ('3', 'ramen', 12)
]

menu_columns = ['product_id', 'product_name', 'price']
menu_df = pd.DataFrame(menu_data, columns = menu_columns)

# Create the "members" table
members_data = [
    ('A', '2021-01-07'),
    ('B', '2021-01-09')
]

members_columns = ['customer_id', 'join_date']
members_df = pd.DataFrame(members_data, columns=members_columns)

In [3]:
print(sales_df.dtypes)
print(menu_df.dtypes)
print(members_df.dtypes)

customer_id    object
order_date     object
product_id     object
dtype: object
product_id      object
product_name    object
price            int64
dtype: object
customer_id    object
join_date      object
dtype: object


-- we need to change order_date and join_date column to datetime

In [4]:
sales_df['order_date'] = pd.to_datetime(sales_df['order_date'], format= '%Y-%m-%d')
members_df['join_date'] = pd.to_datetime(members_df['join_date'], format= '%Y-%m-%d')

-- check created tables

In [5]:
# Print the created DataFrames
print("Sales table:")
print(sales_df)
print("\nMenu table:")
print(menu_df)
print("\nMembers table:")
print(members_df)

Sales table:
   customer_id order_date product_id
0            A 2021-01-01          1
1            A 2021-01-01          2
2            A 2021-01-07          2
3            A 2021-01-10          3
4            A 2021-01-11          3
5            A 2021-01-11          3
6            B 2021-01-01          2
7            B 2021-01-02          2
8            B 2021-01-04          1
9            B 2021-01-11          1
10           B 2021-01-16          3
11           B 2021-02-01          3
12           C 2021-01-01          3
13           C 2021-01-01          3
14           C 2021-01-07          3

Menu table:
  product_id product_name  price
0          1        sushi     10
1          2        curry     15
2          3        ramen     12

Members table:
  customer_id  join_date
0           A 2021-01-07
1           B 2021-01-09


In [6]:
full_data = sales_df.merge(members_df, on = 'customer_id', how = 'left')
full_data = full_data.merge(menu_df, on = 'product_id', how = 'left')
full_data

,customer_id,order_date,product_id,join_date,product_name,price
0,A,2021-01-01,1,2021-01-07,sushi,10
1,A,2021-01-01,2,2021-01-07,curry,15
2,A,2021-01-07,2,2021-01-07,curry,15
3,A,2021-01-10,3,2021-01-07,ramen,12
4,A,2021-01-11,3,2021-01-07,ramen,12
5,A,2021-01-11,3,2021-01-07,ramen,12
6,B,2021-01-01,2,2021-01-09,curry,15
7,B,2021-01-02,2,2021-01-09,curry,15
8,B,2021-01-04,1,2021-01-09,sushi,10
9,B,2021-01-11,1,2021-01-09,sushi,10


 1. What is the total amount each customer spent at the restaurant?

In [7]:
customer_expenditure = full_data.groupby(['customer_id'])['price'].agg(total_amount='sum')
customer_expenditure.reset_index()

,customer_id,total_amount
0,A,76
1,B,74
2,C,36


2. How many days has each customer visited the restaurant?

In [8]:
visited_days = full_data.groupby('customer_id')['order_date'].agg(visted_days='nunique')
visited_days.reset_index()

,customer_id,visted_days
0,A,4
1,B,6
2,C,2


3. What was the first item from the menu purchased by each customer?

In [9]:
full_date = full_data.sort_values('order_date')

In [10]:
first_purchased_item = full_date.groupby('customer_id')['product_name'].first()
first_purchased_item.reset_index()

,customer_id,product_name
0,A,sushi
1,B,curry
2,C,ramen


4. What is the most purchased item on the menu and how many times was it purchased by all customers?

In [11]:
most_item = full_data.groupby(['product_name']).agg(ordered_times=('product_name', 'count')).sort_values('ordered_times', ascending = False).reset_index()
most_item = most_item.sort_values(['ordered_times'], ascending=[False])
most_item.head(1)

,product_name,ordered_times
0,ramen,8


5. Which item was the most popular for each customer?

In [12]:
count_items = full_data.groupby(['customer_id', 'product_name'])['product_name'].agg(ordered_times='count').reset_index()
most_item = count_items.sort_values('ordered_times', ascending=False).groupby(['customer_id','ordered_times'])['product_name'].apply(list).reset_index()
most_item = pd.DataFrame(most_item)
most_item = most_item.groupby('customer_id').first()
most_item

,ordered_times,product_name
customer_id,,
A,1,[sushi]
B,2,"[curry, ramen, sushi]"
C,3,[ramen]


6. Which item was purchased first by the customer after they became a member?

In [13]:
after_member_table = full_data[full_data['order_date'] >= full_data['join_date']].sort_values('order_date', ascending=False)
after_member_table = after_member_table.groupby(['customer_id','order_date'])['product_name'].apply(list).reset_index()
after_member_table = pd.DataFrame(after_member_table)
after_member_table = after_member_table.groupby('customer_id')[['order_date','product_name']].first().reset_index()
after_member_table

,customer_id,order_date,product_name
0,A,2021-01-07,[curry]
1,B,2021-01-11,[sushi]


7. Which item was purchased just before the customer became a member?

In [14]:
before_member_table = full_data[full_data['order_date'] < full_data['join_date']].sort_values('order_date', ascending=False)
before_member_table = before_member_table.groupby(['customer_id','order_date'])['product_name'].apply(list).reset_index()
before_member_table = pd.DataFrame(before_member_table)
before_member_table = before_member_table.groupby('customer_id')[['order_date','product_name']].first().reset_index()
before_member_table

,customer_id,order_date,product_name
0,A,2021-01-01,"[sushi, curry]"
1,B,2021-01-01,[curry]


8. What is the total items and amount spent for each member before they became a member?

In [15]:
spending_before_members_date = full_data.loc[full_data['order_date'] < full_data['join_date']].sort_values('order_date', ascending=False)
spending_before_members_date = spending_before_members_date.groupby(['customer_id']).agg({'product_name': 'count','price': 'sum'})
spending_before_members_date.columns = ['ordered_times', 'total_amount']
spending_before_members_date.reset_index()

,customer_id,ordered_times,total_amount
0,A,2,25
1,B,3,40


9.  If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In [16]:
full_data['points'] = np.where(full_data['product_name']=='sushi', full_data['price']*20,full_data['price']*10)
total_points = full_data[['customer_id','points']].groupby('customer_id').sum()
total_points.reset_index()

,customer_id,points
0,A,860
1,B,940
2,C,360


10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [18]:
event_point=full_data[['customer_id','order_date', 'product_name', 'join_date','price']].copy(deep=True)
condition_1 = (event_point['order_date'] <= pd.to_datetime('2021-01-31'))
condition_2 = (event_point['join_date'].notnull())

In [20]:
filter_event_point=event_point[condition_1 & condition_2].copy(deep=True)
condition_3 = (filter_event_point['order_date']>= filter_event_point['join_date'])
condition_4 = (filter_event_point['order_date'] <= filter_event_point['join_date']+pd.Timedelta(days=6))
normal_point = np.where(filter_event_point['product_name']=='sushi', filter_event_point['price']*20,filter_event_point['price']*10)
filter_event_point['event_points']=np.where(condition_3 & condition_4,filter_event_point['price']*20,normal_point)
filter_event_point[['customer_id','event_points']].groupby('customer_id').sum().reset_index()

,customer_id,event_points
0,A,1370
1,B,820
